In [1]:
import time, datetime, sys
dateString = time.strftime("Y%YM%mD%d")
timeString = time.strftime("UTC %H:%M")
start = datetime.datetime.now()
print(dateString,timeString)
sys.version

Y2017M10D11 UTC 17:17


'3.5.4 |Continuum Analytics, Inc.| (default, Aug 14 2017, 13:26:58) \n[GCC 4.4.7 20120313 (Red Hat 4.4.7-1)]'

In [2]:
INPUT_VERSION = 18
OUTPUT_VERSION = 9

S3_INPUT_PATH_EE  = "s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/"
S3_OUTPUT_PATH = "s3://wri-projects/Aqueduct30/processData/Y2017M10D11_RH_test_routing_V01/output/"

S3_INPUT_PATH_HYDROBASINS = "s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/"

EC2_INPUT_PATH = "/volumes/data/Y2017M10D11_RH_test_routing_V01/input"
EC2_OUTPUT_PATH = "/volumes/data/Y2017M10D11_RH_test_routing_V01/output"

INPUT_FILENAME_EE =  "mergedZonalStatsEE_V%0.2d.pkl" %(INPUT_VERSION)
INPUT_FILENAME_HYDROBASINS =  "hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl"

OUTPUT_FILENAME = "Y2017M10D11_RH_test_routing_V%0.2d" %(OUTPUT_VERSION)

In [3]:
!rm -r {EC2_INPUT_PATH} 
!rm -r {EC2_OUTPUT_PATH} 

In [4]:
!mkdir -p {EC2_INPUT_PATH} 
!mkdir -p {EC2_OUTPUT_PATH} 

In [5]:
!aws s3 cp {S3_INPUT_PATH_EE} {EC2_INPUT_PATH} --recursive

download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V18.pkl to ../../../../data/Y2017M10D11_RH_test_routing_V01/input/mergedZonalStatsEE_V18.pkl
download: s3://wri-projects/Aqueduct30/processData/Y2017M09D14_RH_merge_EE_results_V01/output/mergedZonalStatsEE_V18.csv to ../../../../data/Y2017M10D11_RH_test_routing_V01/input/mergedZonalStatsEE_V18.csv


In [6]:
!aws s3 cp {S3_INPUT_PATH_HYDROBASINS} {EC2_INPUT_PATH} --recursive --exclude "*" --include "*.pkl"

download: s3://wri-projects/Aqueduct30/processData/Y2017M08D29_RH_Merge_FAONames_Upstream_V01/output/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl to ../../../../data/Y2017M10D11_RH_test_routing_V01/input/hybas_lev06_v1c_merged_fiona_upstream_downstream_FAO_V01.pkl


In [7]:
import os
import pandas as pd
import multiprocessing as mp
import pickle
import numpy as np
import itertools
import logging
import pprint
import ast

In [8]:
inputLocationEE = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_EE)
inputLocationHydroBasins = os.path.join(EC2_INPUT_PATH,INPUT_FILENAME_HYDROBASINS)

In [9]:
df_ee = pd.read_pickle(inputLocationEE)

In [10]:
df_ee.index.names = ['PFAF_ID']

In [11]:
df_HydroBasins = pd.read_pickle(inputLocationHydroBasins)

In [12]:
df_complete = df_HydroBasins.merge(df_ee,how="left",left_index=True, right_index=True)

In [13]:
df_complete = df_complete.drop_duplicates(subset='PFAF_ID', keep='first')

In [14]:
df_complete.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,total_volume_PIrrWW_monthY2014M09,count_runoff_monthY2014M10,mean_runoff_monthY2014M10,total_volume_runoff_monthY2014M10,count_PIndWW_monthY2014M01,mean_PIndWW_monthY2014M01,total_volume_PIndWW_monthY2014M01,count_PLivWW_monthY2014M09,mean_PLivWW_monthY2014M09,total_volume_PLivWW_monthY2014M09
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,1060000010,6675,1060000010,0,1060000010,1060000010,0.0,0.0,1890.8,1890.8,...,347598.275922,2479.0,1.989458e-05,37519.523523,2536.0,0.002146,4.048026e+06,2536.0,1.366897e-07,257.785492
111012,1060000100,6676,1060000100,0,1060000100,1060000100,0.0,0.0,2925.9,2925.9,...,0.000000,3914.0,5.720039e-06,16735.674093,3921.0,0.000229,6.688613e+05,3921.0,6.624970e-07,1938.331951
111013,1060000110,6677,1060000110,0,1060000110,1060000110,0.0,0.0,893.5,893.5,...,0.000000,1141.0,4.924584e-06,4394.811805,1194.0,0.000694,6.191623e+05,1194.0,5.123168e-08,45.720323
111014,1060000150,6678,1060000150,0,1060000150,1060000150,0.0,0.0,4217.3,4217.4,...,0.000000,5605.0,0.000000e+00,0.000000,5605.0,0.000018,7.558842e+04,5605.0,2.111498e-07,888.152682
111015,1060000160,6679,1060000160,0,1060000160,1060000160,0.0,0.0,16638.1,16638.1,...,0.000000,21086.0,2.064860e-08,342.706117,21769.0,0.000541,8.983459e+06,21769.0,2.690531e-08,446.549220


In [24]:
df_part_temp2 = df_complete.copy()

In [25]:
df_part_temp2["rutger"] = df_part_temp2["Downstream_PFAF_IDs"].apply(lambda x: ast.literal_eval(x))

In [35]:
df_part_temp2["numberUpstream"] = df_part_temp2["Downstream_PFAF_IDs"].apply(lambda x: len(ast.literal_eval(x)))

In [41]:
df_part_temp2

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,count_PIndWW_monthY2014M01,mean_PIndWW_monthY2014M01,total_volume_PIndWW_monthY2014M01,count_PLivWW_monthY2014M09,mean_PLivWW_monthY2014M09,total_volume_PLivWW_monthY2014M09,rutger,number,numberUpstream,test
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,1060000010,6675,1060000010,0,1060000010,1060000010,0.0,0.0,1890.8,1890.8,...,2536.0,2.146450e-03,4.048026e+06,2536.0,1.366897e-07,257.785492,[],0,0,42.0
111012,1060000100,6676,1060000100,0,1060000100,1060000100,0.0,0.0,2925.9,2925.9,...,3921.0,2.286082e-04,6.688613e+05,3921.0,6.624970e-07,1938.331951,[],0,0,42.0
111013,1060000110,6677,1060000110,0,1060000110,1060000110,0.0,0.0,893.5,893.5,...,1194.0,6.937993e-04,6.191623e+05,1194.0,5.123168e-08,45.720323,[],0,0,42.0
111014,1060000150,6678,1060000150,0,1060000150,1060000150,0.0,0.0,4217.3,4217.4,...,5605.0,1.797042e-05,7.558842e+04,5605.0,2.111498e-07,888.152682,[],0,0,42.0
111015,1060000160,6679,1060000160,0,1060000160,1060000160,0.0,0.0,16638.1,16638.1,...,21769.0,5.412679e-04,8.983459e+06,21769.0,2.690531e-08,446.549220,[],0,0,42.0
111016,1060000790,6680,1060000790,0,1060000790,1060000790,0.0,0.0,1832.5,1832.5,...,2382.0,1.317245e-07,2.410338e+02,2382.0,1.124113e-08,20.569389,[],0,0,42.0
111017,1060000800,6681,1060000800,0,1060000800,1060000800,0.0,0.0,8646.8,8646.8,...,11168.0,7.759812e-05,6.704868e+05,11168.0,3.377294e-08,291.815205,[],0,0,42.0
111018,1060001080,6682,1060001080,0,1060001080,1060001080,0.0,0.0,1955.7,1955.7,...,2509.0,7.704145e-07,1.506817e+03,2509.0,9.598025e-09,18.772324,[],0,0,42.0
111019,1060001090,6683,1060001090,0,1060001090,1060001090,0.0,0.0,6566.3,6566.3,...,8353.0,3.134649e-05,2.053776e+05,8353.0,4.187193e-09,27.433875,[],0,0,42.0


In [ ]:
# Upstream runoff = max(0,local runoff + upstream runoff)
def calculateUpstream(df,temporalResolution):
    
    

In [43]:
def naturalRunoff(localRunoff,upstreamRunoff):
    max(0,localRunoff+upstreamRunoff)
    return x * y

In [ ]:
def availableRunoff(localRunoff,upstreamRunoff,upstreamConsumption):
    
    
    

In [49]:
for numberUpstream in range(0,df_part_temp2["numberUpstream"].max()+1):
    df_part_temp2.loc[df_part_temp2["numberUpstream"] == 0, "test"] = df_part_temp2["total_volume_runoff_monthY2014M10"] + 

SyntaxError: invalid syntax (<ipython-input-49-8b28b7bd1e96>, line 2)

In [45]:
df = df_part_temp2.copy()

In [46]:
df['newcolumn'] = df.apply(lambda x: fxy(x['count_PLivWW_monthY2014M09'], x['count_PLivWW_monthY2014M10']), axis=1)

In [47]:
df.head()

,HYBAS_ID2,Unnamed: 0,HYBAS_ID,NEXT_DOWN,NEXT_SINK,MAIN_BAS,DIST_SINK,DIST_MAIN,SUB_AREA,UP_AREA,...,mean_PIndWW_monthY2014M01,total_volume_PIndWW_monthY2014M01,count_PLivWW_monthY2014M09,mean_PLivWW_monthY2014M09,total_volume_PLivWW_monthY2014M09,rutger,number,numberUpstream,test,newcolumn
PFAF_ID,,,,,,,,,,,,,,,,,,,,,
111011,1060000010,6675,1060000010,0,1060000010,1060000010,0.0,0.0,1890.8,1890.8,...,0.002146,4.048026e+06,2536.0,1.366897e-07,257.785492,[],0,0,37519.523523,6431296.0
111012,1060000100,6676,1060000100,0,1060000100,1060000100,0.0,0.0,2925.9,2925.9,...,0.000229,6.688613e+05,3921.0,6.624970e-07,1938.331951,[],0,0,16735.674093,15374241.0
111013,1060000110,6677,1060000110,0,1060000110,1060000110,0.0,0.0,893.5,893.5,...,0.000694,6.191623e+05,1194.0,5.123168e-08,45.720323,[],0,0,4394.811805,1425636.0
111014,1060000150,6678,1060000150,0,1060000150,1060000150,0.0,0.0,4217.3,4217.4,...,0.000018,7.558842e+04,5605.0,2.111498e-07,888.152682,[],0,0,0.000000,31416025.0
111015,1060000160,6679,1060000160,0,1060000160,1060000160,0.0,0.0,16638.1,16638.1,...,0.000541,8.983459e+06,21769.0,2.690531e-08,446.549220,[],0,0,342.706117,473889361.0


In [16]:
upstreamCatchments = df_part_temp2.loc[111014, "Downstream_PFAF_IDs"]

In [17]:
upstreamCatchments = ast.literal_eval(upstreamCatchments)

In [18]:
len(upstreamCatchments)

0

In [20]:
i = 0 